In [37]:
#Imports
import pandas as pd
import numpy as np 
import nilearn 
import nibabel as nib
import os
import matplotlib.pyplot as plt
import subprocess


from nilearn import plotting, image
from nilearn.glm.first_level import compute_regressor
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_design_matrix
from numpy import savetxt
from subprocess import call
from subprocess import Popen

#Submit job to HPC
#subprocess.call(['qsub', '-l', 'procs=1,mem=24gb,walltime=12:00:00 -I'])

# Init variables
init_folder='/home/traaffneu/margal/code/multirat_se/script/'
analysis_folder='/project/4180000.19/multirat_stim/rabies/preprocess'
            
# Data path
template_path ='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
metadata_path = '/home/traaffneu/margal/code/multirat_se/script/table/metadata_stand.tsv'

# Paths for the directories
output_dir = '/project/4180000.19/multirat_stim/rabies_test/first_level_analysis/'
matrix_dir = os.path.join(output_dir, 'matrix/')
image_dir = os.path.join(output_dir, 'image/')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(matrix_dir):
    os.makedirs(matrix_dir)
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
    


In [25]:
df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]

print(df['func.sensory.onset'])

In [39]:
 # ---------- FIRST LEVEL ANALYSIS ------------

df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]

for index in range(47, 48):
    
    #sibmit job to HPC
    p1 = subprocess.Popen(["echo", f"{init_folder}"], stdout=subprocess.PIPE)
    subprocess.call(["qsub", "-N", 'glm', "-l", 'nodes=1:ppn=1,mem=12gb,walltime=00:10:00'], stdin=p1.stdout)        #subprocess.call(["./RABIES_preprocess.sh", subj_num, TR, correction_arg], shell=False) #run RABIES_preprocess, giving additional inputs: correction_args, specified in metadat_rabies, submit job to qsub

    subj_num=str(df.iloc[index]['rat.sub'])[:-2]
    print(subj_num)

    subject_path = analysis_folder+'/sub-0{}_ses-1/bold_datasink/commonspace_bold/_scan_info_subject_id0{}.session1_split_name_sub-0{}_ses-1_T2w/_run_1/sub-0{}_ses-1_run-1_bold_combined.nii.gz'.format(subj_num, subj_num, subj_num, subj_num)

    tr=(df.iloc[index]['func.TR'])
    hrf_model = 'spm'
    func_img = nib.load(subject_path)
    header = func_img.header


    dimension = np.array(image.load_img(subject_path).shape)          #put in an array; dimension, number of slices, number of volumes 
    n_scans = dimension[3]                                            #nb volumes
    frame_times = np.arange(n_scans) * tr                             # corresponding frame times

    # --- Events regressor --- 
    
    onset = np.matrix(df.iloc[index]['func.sensory.onset']).A[0]
    duration = np.matrix(df.iloc[index]['func.sensory.duration']).A[0]
    events = pd.DataFrame({'onset': onset,'duration': duration})
    events_list = events.append(events)

    print(df.iloc[index]['func.sensory.onset'])
    
    # --- Motion regressors ---

    cofounders_file = 'sub-0{}_ses-1_run-1_bold_RAS_combined_confounds.csv'.format(subj_num)
    cofounders_path = analysis_folder+'/sub-0{}_ses-1/confounds_datasink/confounds_csv/_scan_info_subject_id0{}.session1_split_name_sub-0{}_ses-1_T2w/_run_1/sub-0{}_ses-1_run-1_bold_combined_confounds.csv'.format(subj_num, subj_num, subj_num, subj_num)

    cofounders = pd.read_csv(cofounders_path, sep=',')

    motion_names = ['mov1', 'mov2', 'mov3', 'rot1', 'rot2', 'rot3']
    motion = np.array(cofounders[motion_names])
    
    
    # --- Compute design matrix --- 

    design_matrix = make_first_level_design_matrix(frame_times=frame_times, 
                                                   events=events, 
                                                   add_regs=motion, 
                                                   add_reg_names=motion_names, 
                                                   drift_model="polynomial", 
                                                   drift_order=3, 
                                                   hrf_model=hrf_model, 
                                                   high_pass=0.01)
    plot_design_matrix(design_matrix) 
    savetxt(matrix_dir+'matrix_sub-0{}_ses-1.csv'.format(subj_num), design_matrix, delimiter=',')

    # --- Fitting a first-level model --- 
    
    fmri_glm = FirstLevelModel()
    fmri_glm = fmri_glm.fit(subject_path, design_matrices=design_matrix)
    
    # -- Coompute contrasts ---
    
    n_columns = design_matrix.shape[1]
    contrast_val = np.hstack(([1], np.zeros(n_columns - 1)))
    print('Contrasts: ', contrast_val)

    statistical_map= fmri_glm.compute_contrast(contrast_val, output_type='all')

    plotting.plot_stat_map(statistical_map['z_score'], bg_img = template_path, threshold = 1.9, title = 'Test: Rat{}'.format(subj_num))
    plt.savefig(image_dir+'stat_map_sub-0{}_ses-1'.format(subj_num)+'.png')


48882169.dccn-l029.dccn.nl
200509


MemoryError: 